<img src="../assets/CoLLIE_blue.png" alt="GoLLIE" width="200"/>

# Custom Tasks with GoLLIE

This notebook is an example of how to run Custom Tasks with GoLLIE. This notebook covers:

- How to define the guidelines for a task
- How to load GoLLIE
- How to generate model inputs
- How to parse the output
- How to implement a scorer and evaluate the output

You can modify this notebook to run any task task you want 

### Import requeriments

See the requeriments.txt file in the main directory to install the required dependencies

In [1]:
import sys
sys.path.append("../") # Add the GoLLIE base directory to sys path

In [21]:
import rich
import logging
from src.model.load_model import load_model
import black
import inspect
from jinja2 import Template as jinja2Template
import tempfile
from src.tasks.utils_typing import AnnotationList
logging.basicConfig(level=logging.INFO)
from typing import Dict, List, Type
import torch

## Load GoLLIE

We will load GOLLIE-7B from the huggingface-hub.
You can use the function AutoModelForCausalLM.from_pretrained if you prefer it. However, we provide a handy load_model function with many functionalities already implemented that will assist you in reproducing our results.

Please note that setting use_flash_attention=True is mandatory. Our flash attention implementation has small numerical differences compared to the attention implementation in Huggingface. Using use_flash_attention=False will result in the model producing inferior results. Flash attention requires an available CUDA GPU. Running GOLLIE pre-trained models on a CPU is not supported. We plan to address this in future releases.

- Set force_auto_device_map=True to automatically load the model on available GPUs.
- Set quantization=4 if the model doesn't fit in your GPU memory.

In [3]:
model, tokenizer = load_model(
    inference=True,
    model_weights_name_or_path="HiTZ/GoLLIE-7B",
    quantization=None,
    use_lora=False,
    force_auto_device_map=True,
    use_flash_attention=True,
    torch_dtype="bfloat16"
)

INFO:root:Loading model model from HiTZ/GoLLIE-7B
INFO:root:We will load the model using the following device map: auto and max_memory: None


config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/945 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

INFO:root:Model dtype: torch.bfloat16
INFO:root:Total model memory footprint: 13477.101762 MB


## Define the guidelines

First, we will define the labels and guidelines for the task. We will represent them as Python classes.

The following guidelines have been defined for this example. They were not part of the pre-training dataset. Therefore, we will run GOLLIE in zero-shot settings using unseen labels.

We will use the `Generic` class, which is a versatile class that allows for the implementation of any task you want. However, since the model has never seen the Generic label during training, we will rename it to Template, which is recognized by the model (as it was used in the Tacred dataset).

We will define two classes: `Launcher` and `Mission`. Each class will have a definition and a set of slots that the model needs to fill. Each slot also requires a type definition and a short description, which can include examples. For instance, for the `Launcher` class, we define three slots:

- The `mention`, which will be the name of the Launcher vehicle and should be a string.
- The `space_company` that operated the vehicle, which will also be a string.
- The `crew`, which is defined as a list of astronauts. Therefore, GoLLIE will fill this slot with a list of strings.

💡 Be creative and try to define your own guidelines to test GoLLIE!

In [4]:
from typing import List

from src.tasks.utils_typing import dataclass
from src.tasks.utils_typing import Generic as Template

"""
Entity definitions
"""


@dataclass
class Launcher(Template):
    """Refers to a vehicle designed primarily to transport payloads from the Earth's 
    surface to space. Launchers can carry various payloads, including satellites, 
    crewed spacecraft, and cargo, into various orbits or even beyond Earth's orbit. 
    They are usually multi-stage vehicles that use rocket engines for propulsion."""

    mention: str  
    """
    The name of the launcher vehicle. 
    Such as: "Sturn V", "Atlas V", "Soyuz", "Ariane 5"
    """
    space_company: str # The company that operates the launcher. Such as: "Blue origin", "ESA", "Boeing", "ISRO", "Northrop Grumman", "Arianespace"
    crew: List[str] # Names of the crew members boarding the Launcher. Such as: "Neil Armstrong", "Michael Collins", "Buzz Aldrin"
    

@dataclass
class Mission(Template):
    """Any planned or accomplished journey beyond Earth's atmosphere with specific objectives, 
    either crewed or uncrewed. It includes missions to satellites, the International 
    Space Station (ISS), other celestial bodies, and deep space."""
    
    mention: str
    """
    The name of the mission. 
    Such as: "Apollo 11", "Artemis", "Mercury"
    """
    date: str # The start date of the mission
    departure: str # The place from which the vehicle will be launched. Such as: "Florida", "Houston", "French Guiana"
    destination: str # The place or planet to which the launcher will be sent. Such as "Moon", "low-orbit", "Saturn"


ENTITY_DEFINITIONS: List[Template] = [
    Launcher,
    Mission,
]
    
if __name__ == "__main__":
    cell_txt = In[-1]

### Print the guidelines to guidelines.py

Due to IPython limitations, we must write the content of the previous cell to a file and then import the content from that file.

In [5]:
with open("guidelines.py","w",encoding="utf8") as python_guidelines:
    print(cell_txt,file=python_guidelines)

from guidelines import *

We use inspect.getsource to get the guidelines as a string

In [10]:
guidelines = [inspect.getsource(definition) for definition in ENTITY_DEFINITIONS]
guidelines

['@dataclass\nclass Launcher(Template):\n    """Refers to a vehicle designed primarily to transport payloads from the Earth\'s \n    surface to space. Launchers can carry various payloads, including satellites, \n    crewed spacecraft, and cargo, into various orbits or even beyond Earth\'s orbit. \n    They are usually multi-stage vehicles that use rocket engines for propulsion."""\n\n    mention: str  \n    """\n    The name of the launcher vehicle. \n    Such as: "Sturn V", "Atlas V", "Soyuz", "Ariane 5"\n    """\n    space_company: str # The company that operates the launcher. Such as: "Blue origin", "ESA", "Boeing", "ISRO", "Northrop Grumman", "Arianespace"\n    crew: List[str] # Names of the crew members boarding the Launcher. Such as: "Neil Armstrong", "Michael Collins", "Buzz Aldrin"\n',
 '@dataclass\nclass Mission(Template):\n    """Any planned or accomplished journey beyond Earth\'s atmosphere with specific objectives, \n    either crewed or uncrewed. It includes missions to s

## Define input sentence

Here we define the input sentence and the gold labels.

You can define and empy list as gold labels if you don't have gold annotations.

In [11]:
text = "The Ares 3 mission to Mars is scheduled for 2032. The Starship rocket build by SpaceX will take off from Boca Chica, carrying the astronauts Max Rutherford, Elena Soto, and Jake Martinez."
gold = [
    Launcher(mention="Starship",space_company="SpaceX",crew=["Max Rutherford","Elena Soto","Jake Martinez"]),
    Mission(mention="Ares 3",date="2032",departure="Boca Chica",destination="Mars")
]

## Filling a template

We need to define a template. For this task, we will include only the class definitions and the text to be annotated. However, you can design different templates to incorporate more information (for example, event triggers, as demonstrated in the Event Extraction notebook).

We will use Jinja templates, which are easy to implement and exceptionally fast. For more information, visit: https://jinja.palletsprojects.com/en/3.1.x/api/#high-level-api.



In [12]:
template_txt =(
"""# The following lines describe the task definition
{%- for definition in guidelines %}
{{ definition }}
{%- endfor %}

# This is the text to analyze
text = {{ text.__repr__() }}

# The annotation instances that take place in the text above are listed here
result = [
{%- for ann in annotations %}
    {{ ann }},
{%- endfor %}
]
""")


In [13]:

template = jinja2Template(template_txt)
# Fill the template
formated_text = template.render(guidelines=guidelines, text=text, annotations=gold, gold=gold)

### Black Code Formatter

We use the Black Code Formatter to automatically unify all the prompts to the same format. 

https://github.com/psf/black

In [14]:
black_mode = black.Mode()
formated_text = black.format_str(formated_text, mode=black_mode)

### Print the filled and formatted template

In [15]:
rich.print(formated_text)

# The following lines describe the task definition
@dataclass
class Launcher(Template):
    """Refers to a vehicle designed primarily to transport payloads from the Earth's
    surface to space. Launchers can carry various payloads, including satellites,
    crewed spacecraft, and cargo, into various orbits or even beyond Earth's orbit.
    They are usually multi-stage vehicles that use rocket engines for propulsion."""

    mention: str
    """
    The name of the launcher vehicle. 
    Such as: "Sturn V", "Atlas V", "Soyuz", "Ariane 5"
    """
    space_company: str  # The company that operates the launcher. Such as: "Blue origin", "ESA", "Boeing", "ISRO", 
"Northrop Grumman", "Arianespace"
    crew: List[
        str
    ]  # Names of the crew members boarding the Launcher. Such as: "Neil Armstrong", "Michael Collins", "Buzz 
Aldrin"


@dataclass
class Mission(Template):
    """Any planned or accomplished journey beyond Earth's atmosphere with specific objectives,
    either crewed or uncrewed. It includes missions to satellites, the International
    Space Station (ISS), other celestial bodies, and deep space."""

    mention: str
    """
    The name of the mission. 
    Such as: "Apollo 11", "Artemis", "Mercury"
    """
    date: str  # The start date of the mission
    departure: str  # The place from which the vehicle will be launched. Such as: "Florida", "Houston", "French 
Guiana"
    destination: str  # The place or planet to which the launcher will be sent. Such as "Moon", "low-orbit", 
"Saturn"


# This is the text to analyze
text = "The Ares 3 mission to Mars is scheduled for 2032. The Starship rocket build by SpaceX will take off from 
Boca Chica, carrying the astronauts Max Rutherford, Elena Soto, and Jake Martinez."

# The annotation instances that take place in the text above are listed here
result = [
    Launcher(
        mention="Starship",
        space_company="SpaceX",
        crew=["Max Rutherford", "Elena Soto", "Jake Martinez"],
    ),
    Mission(mention="Ares 3", date="2032", departure="Boca Chica", destination="Mars"),
]

## Prepare model inputs

We remove everything after `result =` to run inference with the model.

In [16]:
prompt, _ = formated_text.split("result =")
prompt = prompt + "result ="

Tokenize the input sentence

In [17]:
model_input = tokenizer(prompt, add_special_tokens=True, return_tensors="pt")

Remove the `eos` token from the input

In [18]:
model_input["input_ids"] = model_input["input_ids"][:, :-1]
model_input["attention_mask"] = model_input["attention_mask"][:, :-1]

## Run GoLLIE

We generate the predictions using GoLLIE.

We use `num_beams=1` and `do_sample=False` in our exmperiments. But feel free to experiment with differen decoding strategies 😊

In [22]:
%%time

device = "cuda:3" if torch.cuda.is_available() else "cpu"

model_ouput = model.generate(
    **model_input.to(model.device),
    max_new_tokens=128,
    do_sample=False,
    min_new_tokens=0,
    num_beams=1,
    num_return_sequences=1,
)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: invalid argument to exchangeDevice

### Print the results

In [38]:
for y, x in enumerate(model_ouput):
    print(f"Answer {y}")
    rich.print(tokenizer.decode(x,skip_special_tokens=True).split("result = ")[-1])

Answer 0


[
    Mission(
        mention="Ares 3",
        date="2032",
        departure="Boca Chica",
        destination="Mars",
    ),
    Launcher(
        mention="Starship",
        space_company="SpaceX",
        crew=["Max Rutherford", "Elena Soto", "Jake Martinez"],
    ),
]

## Parse the output

The output is a Python list of instances, we can execute it  🤯

We define the AnnotationList class to parse the output with a single line of code. The `AnnotationList.from_output` function filters any label that we did not define (hallucinations) to prevent getting an `undefined class` error. 

In [39]:
result = AnnotationList.from_output(
    tokenizer.decode(model_ouput[0],skip_special_tokens=True).split("result = ")[-1],
    task_module="guidelines"
    )
rich.print(result)

[
    Mission(mention='Ares 3', date='2032', departure='Boca Chica', destination='Mars'),
    Launcher(mention='Starship', space_company='SpaceX', crew=['Max Rutherford', 'Elena Soto', 'Jake Martinez'])
]

Labels are an instance of the defined classes:

In [40]:
type(result[0])

guidelines.Mission

In [41]:
result[0].mention

'Ares 3'

# Evaluate the result

Finally, we will evaluate the outputs from the model.

First, we define an Scorer, for Named Entity Recognition, we will use the `SpanScorer` class.

We need to define the `valid_types` for the scorer, which will be the labels that we have defined. 

In [42]:
from src.tasks.utils_scorer import TemplateScorer

class MyScorer(TemplateScorer):
    """Compute the F1 score for Generic Task"""

    valid_types: List[Type] = ENTITY_DEFINITIONS

### Instanciate the scorer

In [43]:
scorer = MyScorer()

### Compute F1 

In [44]:
scorer_results = scorer(reference=[gold],predictions=[result])
rich.print(scorer_results)

{
    'templates': {
        'precision': 1.0,
        'recall': 1.0,
        'f1-score': 1.0,
        'class_scores': {
            'Launcher': {
                'tp': 1,
                'total_pos': 1,
                'total_pre': 1,
                'precision': 1.0,
                'recall': 1.0,
                'f1-score': 1.0
            },
            'Mission': {
                'tp': 1,
                'total_pos': 1,
                'total_pre': 1,
                'precision': 1.0,
                'recall': 1.0,
                'f1-score': 1.0
            }
        }
    },
    'slots': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0}
}

GoLLIE has successfully labeled a sentence using a set of labels that were not part of the pretraining dataset 🎉🎉🎉

GoLLIE will perform well on labels with well-defined and clearly bounded guidelines. 

Please share your cool experiments with us; we'd love to see what everyone is doing with GoLLIE!
- [@iker_garciaf](https://twitter.com/iker_garciaf)
- [@osainz59](https://twitter.com/osainz59)